# Tensorflow 2 Object Detection: Train model



## Installation

First, verify if GPU is available for training. If not, please adjust this in your environment (in Colab).

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Download tensorflow models repo including the TFOD API

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Download our repository

In [ ]:

if "animal-detection" in pathlib.Path.cwd().parts:
  while "animal-detection" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('animal-detection').exists():
  !git clone --depth 1 https://github.com/Philideli/animal-breed-detector.git ./animal-detection

Adjust setup.py according to answer in https://github.com/tensorflow/tensorflow/issues/64349
this is a temporary workaround and is an issue with tf/keras

In [ ]:

!rm /content/models/research/object_detection/packages/tf2/setup.py
!cp /content/animal-detection/src/hotfixes/object_detection_tf2_hotfix_setup.py /content/models/research/object_detection/packages/tf2/setup.py

Temporary hack for tf 2.15 - adjust code in tfexample_decoder inside of the tf_slim package

In [ ]:

!rm /usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py
!cp /content/animal-detection/src/hotfixes/tfexample_decoder_tf_slim_lib_hotfix.py /usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py

Install the Object Detection API

In [ ]:

%%bash
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Run model builder test. Only continue if the test was successful

In [ ]:

!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

## Prepare data

In [ ]:
%cd /content/animal-detection/src

Download the train/test data: images of animals and annotations with their breeds and other info

In [ ]:
import urllib, os
urllib.request.urlretrieve("https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz", "annotations.tar.gz")
print('downloaded annotations')
urllib.request.urlretrieve("https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz", "images.tar.gz")
print('downloaded images')

os.makedirs('./data/datasets/raw/my_dataset/annotations', exist_ok=True)
os.makedirs('./data/datasets/raw/my_dataset/images', exist_ok=True)
!tar -zxf ./annotations.tar.gz --directory ./data/datasets/raw/my_dataset/annotations
!tar -zxf ./images.tar.gz --directory ./data/datasets/raw/my_dataset/images

In [ ]:
%cd /content/animal-detection

In [ ]:
! python -m src.data.preparation -m verbose -sr 0.2

### Retrieve generate_tfrecord.py and labelmap
There are multiple ways to do this, I have included the files needed within my Kaggle dataset, but you could also download them from Github, or even connect your Google Drive and retrieve them that way

In [ ]:
train_record_path = '/content/animal-detection/src/data/datasets/processed/my_dataset/train.record'
test_record_path = '/content/animal-detection/src/data/datasets/processed/my_dataset/eval.record'
labelmap_path = '/content/animal-detection/src/data/datasets/processed/my_dataset/label_map.pbtxt'

## Configuring training

Now that the data is ready it's time to create a training configuration. The OD API supports lots of models, each with its own config file. In this notebook I'm making use of EfficientDet, but you can replace it with any model available in the [Tensorflow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md).

In [ ]:
%cd /content/animal-detection/src/models

In [ ]:
batch_size = 12
num_steps = 30000
num_eval_steps = 5000
num_classes = 37

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
!tar -xf efficientdet_d0_coco17_tpu-32.tar.gz

In [ ]:
%cd /content/animal-detection/src/models/efficientdet_d0_coco17_tpu-32

In [ ]:
fine_tune_checkpoint = '/content/animal-detection/src/models/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0'

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d0_512x512_coco17_tpu-8.config

base_config_path = '/content/animal-detection/src/models/efficientdet_d0_coco17_tpu-32/ssd_efficientdet_d0_512x512_coco17_tpu-8.config'

In [ ]:
%cd /content/animal-detection/src/models
!mkdir custom-model

In [ ]:
%cd /content/animal-detection/src/models/custom-model

In [ ]:
# edit configuration file (from https://colab.research.google.com/drive/1sLqFKVV94wm-lglFq_0kGo2ciM0kecWD)

import re

with open(base_config_path) as f:
    config = f.read()

with open('model_config.config', 'w') as f:

  # Set labelmap path
  config = re.sub('label_map_path: ".*?"',
             'label_map_path: "{}"'.format(labelmap_path), config)

  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)

  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
                  'input_path: "{}"'.format(train_record_path), config)

  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")',
                  'input_path: "{}"'.format(test_record_path), config)

  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(num_classes), config)

  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)

  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)

  # Set fine-tune checkpoint type to detection
  config = re.sub('fine_tune_checkpoint_type: "classification"',
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)

  f.write(config)

In [ ]:
%cat /content/animal-detection/src/models/custom-model/model_config.config

In [ ]:
model_dir = '/content/animal-detection/src/models/custom-model/'
pipeline_config_path = '/content/animal-detection/src/models/custom-model/model_config.config'

## Train detector

In [ ]:
# Run this block first and use the refresh arrow that will appear in the header once the training below starts
%load_ext tensorboard
%tensorboard --logdir '/content/animal-detection/src/models/custom-model/train'

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

## Export model inference graph

In [ ]:
output_directory = '/content/animal-detection/src/models/custom-model/inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

### Download model

In [ ]:
# Zip and download your new model to your system
from google.colab import files
!zip -r /content/animal-detection/src/models/custom-model/model.zip /content/animal-detection/src/models/custom-model
files.download(f'/content/animal-detection/src/models/custom-model/model.zip')

In [ ]:
# Optional: save a copy of the training data to your drive in case you want to re-train later
from google.colab import drive
drive.mount('/content/drive')
!cp -r training/ drive/MyDrive/